# Install required packages 

In [1]:
# install required packages for the project

%pip install torch torchvision torchaudio -quit --index-url https://download.pytorch.org/whl/cpu
%pip install -quit matplotlib
%pip install -quit safetensors


Usage:   
  /home/ram/Desktop/DeepLearning/env/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /home/ram/Desktop/DeepLearning/env/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /home/ram/Desktop/DeepLearning/env/bin/python -m pip install [options] [-e] <vcs project url> ...
  /home/ram/Desktop/DeepLearning/env/bin/python -m pip install [options] [-e] <local project path> ...
  /home/ram/Desktop/DeepLearning/env/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.

Usage:   
  /home/ram/Desktop/DeepLearning/env/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /home/ram/Desktop/DeepLearning/env/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /home/ram/Desktop/DeepLearning/env/bin/python -m pip install [options] [-e] <vcs project url>

##   import necessary libraries

In [ ]:
# import necessary libraries
import time
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import matplotlib.pyplot as plt